## POS tagging using modified Viterbi

# --------------------------------------------------------------------------------------------

# ******************* Syntactic Processing Assignment ********************************

### **********************    Part 1 - HMMs and Viterbi algorithm for POS tagging **********************************************

### Author: Sushma KR ############
### Date: 27th July 2018

# --------------------------------------------------------------------------------------------

### Data Preparation

In [1]:
#Importing libraries
import random

import numpy as np
import pandas as pd

import time

import re

import nltk
from nltk import word_tokenize 
nltk.download('universal_tagset')

import sklearn
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/raagzcd/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [2]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [3]:
len(nltk_data)

3914

In [4]:
# first few tagged sentences
print(nltk_data[:30])

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], [('Rudolph', 'NOUN'), ('Agnew', 'NOUN'), (',', '.'), ('55', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Consolidated', 'NOUN'), ('Gold', 'NOUN'), ('Fields', 'NOUN'), ('PLC', 'NOUN'), (',', '.'), ('was', 'VERB'), ('named', 'VERB'), ('*-1', 'X'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('British', 'ADJ'), ('industrial', 'ADJ'), ('

In [5]:
# Splitting into training and test data
random.seed(1234)
train_set, test_set = train_test_split(nltk_data, train_size = 0.95, random_state = 42)

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [6]:
# Get list of all words along with the POS tags assigned to each word
train_tagged_words = [tup for sent in train_set for tup in sent]

In [7]:
# Print number of word, POS tag combinations obtained
print(len(train_tagged_words))

95589


In [8]:
# list of all words
[tup[0] for tup in train_tagged_words]

['Bank',
 'of',
 'New',
 'England',
 "'s",
 'shares',
 'are',
 'traded',
 '*-1',
 'on',
 'the',
 'New',
 'York',
 'Stock',
 'Exchange',
 '.',
 '$',
 '130',
 'million',
 '*U*',
 'of',
 'general',
 'obligation',
 'distributable',
 'state',
 'aid',
 'bonds',
 'due',
 '1991-2000',
 'and',
 '2009',
 ',',
 'tentatively',
 'priced',
 '*',
 'by',
 'a',
 'Chemical',
 'Securities',
 'Inc.',
 'group',
 '*',
 'to',
 'yield',
 'from',
 '6.20',
 '%',
 'in',
 '1991',
 'to',
 '7.272',
 '%',
 'in',
 '2009',
 '.',
 '``',
 'They',
 'were',
 'a',
 'self-perpetuating',
 'club',
 'that',
 '*T*-229',
 'treated',
 'the',
 'tower',
 'as',
 'sort',
 'of',
 'a',
 'separate',
 'premises',
 ',',
 "''",
 'the',
 'Vicar',
 'Hummerstone',
 'says',
 '*T*-1',
 '.',
 'Valley',
 'Federal',
 'Savings',
 '&',
 'Loan',
 'Association',
 'took',
 'an',
 '$',
 '89.9',
 'million',
 '*U*',
 'charge',
 'as',
 'it',
 'reported',
 'a',
 'third-quarter',
 'loss',
 'of',
 '$',
 '70.7',
 'million',
 '*U*',
 ',',
 'or',
 '$',
 '12.09',

In [9]:
# Unique words comprising the vocabulary
V = set([tup[0] for tup in train_tagged_words])
print(len(V))

12109


In [10]:
# Unique POS tags assigned in the words in the universal penn tree bank
T = set([tup[1] for tup in train_tagged_words])

In [11]:
# List of all POS tags
print(T)

{'X', 'ADJ', 'DET', 'VERB', 'NOUN', 'ADP', '.', 'PRON', 'ADV', 'CONJ', 'PRT', 'NUM'}


### Emission Probabilities

In [12]:
# Return 
# 1. the number of times a word appears given a certain POS tag
# 2. Number of words having the given POS tag
def word_given_tag(word, pos_tag, train_bag = train_tagged_words):
    word_tag_combo = [tup for tup in train_bag if tup[1] == pos_tag]
    return(len([tup for tup in word_tag_combo if tup[0] == word]), len(word_tag_combo))

In [13]:
# Below is an example for an imperative verb. Such verbs are typically at the beginning of a sentence and used for 
# ordering

# Since the training data is unlikely to have the word or exists with a low probability, the probability of the correct
# POS tag getting selected is low

In [14]:
word_given_tag('Show', 'NOUN')

(1, 27423)

In [15]:
word_given_tag('Show', 'VERB')

(0, 12885)

In [16]:
word_given_tag('Brunswick', 'NOUN')

(1, 27423)

### Transition Probabilities

In [17]:
# Transition probabilities are assigned based on the likelihood of a word taking up a certain POS tag by virtue of 
# the prev word POS tag

In [18]:
def tag2_given_tag1(tag2, tag1, train_bag = train_tagged_words):
    tags = [tup[1] for tup in train_bag]
    count_tag2 = len([t for t in tags if t == tag1])
    count_tag2_given_tag1 = 0
    for index in range(len(tags)-1):
        if tags[index] == tag1 and tags[index+1] == tag2:
            count_tag2_given_tag1 += 1
    return(count_tag2_given_tag1, count_tag2)        

In [19]:
print(tag2_given_tag1(tag2='NOUN', tag1='DET'))

(5302, 8284)


### Capture transition probabilities in a t x t matrix

In [20]:
#### Create t x t transition matrix of tags to capture the transition probabilities
tags_matrix = np.zeros((len(T), len(T)), dtype='float32')

In [21]:
for index1, tag1 in enumerate(T):
    for index2, tag2 in enumerate(T):
        tag2_given_tag1_cnts = tag2_given_tag1(tag2, tag1)
        tags_matrix[index1, index2] = tag2_given_tag1_cnts[0]/ tag2_given_tag1_cnts[1]

In [22]:
tags_matrix

array([[7.64818341e-02, 1.65710635e-02, 5.51306568e-02, 2.03632891e-01,
        6.13448061e-02, 1.42925426e-01, 1.63798600e-01, 5.60866781e-02,
        2.51752716e-02, 1.03569152e-02, 1.85787126e-01, 2.70873168e-03],
       [2.13921349e-02, 6.66447282e-02, 5.10120112e-03, 1.23416157e-02,
        6.96725368e-01, 7.89863393e-02, 6.53282851e-02, 6.58219506e-04,
        4.60753683e-03, 1.69491526e-02, 1.08606219e-02, 2.04048045e-02],
       [4.55094166e-02, 2.04973444e-01, 5.31144394e-03, 3.83872539e-02,
        6.40028954e-01, 9.05359723e-03, 1.79864801e-02, 3.74215352e-03,
        1.23128919e-02, 4.82858537e-04, 2.41429268e-04, 2.19700634e-02],
       [2.18005434e-01, 6.46488145e-02, 1.33100510e-01, 1.69188976e-01,
        1.10904150e-01, 9.04928222e-02, 3.53123769e-02, 3.62436958e-02,
        8.25766400e-02, 5.58789307e-03, 3.11214589e-02, 2.28172299e-02],
       [2.91361269e-02, 1.22889541e-02, 1.33099956e-02, 1.47977978e-01,
        2.64631867e-01, 1.76275387e-01, 2.39178792e-01, 4.92

In [23]:
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index = list(T))

In [24]:
tags_df

,X,ADJ,DET,VERB,NOUN,ADP,.,PRON,ADV,CONJ,PRT,NUM
X,0.076482,0.016571,0.055131,0.203633,0.061345,0.142925,0.163799,0.056087,0.025175,0.010357,0.185787,0.002709
ADJ,0.021392,0.066645,0.005101,0.012342,0.696725,0.078986,0.065328,0.000658,0.004608,0.016949,0.010861,0.020405
DET,0.045509,0.204973,0.005311,0.038387,0.640029,0.009054,0.017986,0.003742,0.012313,0.000483,0.000241,0.021970
VERB,0.218005,0.064649,0.133101,0.169189,0.110904,0.090493,0.035312,0.036244,0.082577,0.005588,0.031121,0.022817
NOUN,0.029136,0.012289,0.013310,0.147978,0.264632,0.176275,0.239179,0.004923,0.016884,0.041936,0.043832,0.009627
ADP,0.034029,0.105297,0.326378,0.008240,0.321776,0.017228,0.039486,0.069128,0.013162,0.000856,0.001498,0.062921
.,0.026623,0.044972,0.173502,0.088505,0.223152,0.091114,0.093812,0.065389,0.052078,0.057924,0.002339,0.080500
PRON,0.092819,0.074866,0.009549,0.480901,0.211230,0.022918,0.041253,0.008021,0.033995,0.004966,0.012223,0.007257
ADV,0.023588,0.130233,0.067110,0.344518,0.030897,0.119601,0.136877,0.015615,0.081063,0.006312,0.013621,0.030565
CONJ,0.007977,0.116847,0.121539,0.153918,0.349132,0.054435,0.034256,0.058658,0.055842,0.000469,0.004693,0.042234


### Build the vanilla Viterbi based POS tagger

In [25]:
# Viterbi Heuristic

def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))

    print(T)
    
    for key, word in enumerate(words):

        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)

            pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [26]:
# Running Viterbi algorithm on test set

In [27]:
# list of tagged words
test_run_base = [tup for sent in test_set for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_set for tup in sent]

In [28]:
start = time.time()
tagged_seq = Viterbi(test_tagged_words[:])
end = time.time()
difference = end-start

['X', 'ADJ', 'DET', 'VERB', 'NOUN', 'ADP', '.', 'PRON', 'ADV', 'CONJ', 'PRT', 'NUM']


In [29]:
print(tagged_seq)

[('For', 'ADP'), ('the', 'DET'), ('Agency', 'NOUN'), ('for', 'ADP'), ('International', 'NOUN'), ('Development', 'NOUN'), (',', '.'), ('appropriators', 'NOUN'), ('approved', 'VERB'), ('$', '.'), ('200', 'NUM'), ('million', 'NUM'), ('*U*', 'X'), ('in', 'ADP'), ('secondary', 'ADJ'), ('loan', 'NOUN'), ('guarantees', 'NOUN'), ('under', 'ADP'), ('an', 'DET'), ('expanded', 'VERB'), ('trade', 'VERB'), ('credit', 'NOUN'), ('insurance', 'NOUN'), ('program', 'NOUN'), (',', '.'), ('and', 'CONJ'), ('total', 'ADJ'), ('loan', 'NOUN'), ('guarantees', 'NOUN'), ('for', 'ADP'), ('the', 'DET'), ('Overseas', 'X'), ('Private', 'ADJ'), ('Investment', 'NOUN'), ('Corp.', 'NOUN'), ('are', 'VERB'), ('increased', 'VERB'), ('*-3', 'X'), ('by', 'ADP'), ('$', '.'), ('40', 'NUM'), ('million', 'NUM'), ('*U*', 'X'), ('over', 'ADP'), ('fiscal', 'ADJ'), ('1989', 'NUM'), ('as', 'ADP'), ('part', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('same', 'ADJ'), ('Poland', 'NOUN'), ('package', 'NOUN'), ('.', '.'), ('The', 'DET'), ('m

In [30]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 

In [31]:
accuracy = len(check)/len(tagged_seq)
print(accuracy)

0.9148810693925693


In [32]:
# Incorrectly tagged cases

In [33]:
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1]]

In [34]:
incorrect_tagged_cases

[[('expanded', 'VERB'), (('trade', 'VERB'), ('trade', 'NOUN'))],
 [('the', 'DET'), (('Overseas', 'X'), ('Overseas', 'NOUN'))],
 [('Overseas', 'NOUN'), (('Private', 'ADJ'), ('Private', 'NOUN'))],
 [('settled', 'VERB'), (('pre-1917', 'X'), ('pre-1917', 'ADJ'))],
 [('``', '.'), (('Unemployment', 'X'), ('Unemployment', 'NOUN'))],
 [('the', 'DET'), (('purchasing', 'NOUN'), ('purchasing', 'VERB'))],
 [('weekly', 'ADJ'), (('paycheck', 'X'), ('paycheck', 'NOUN'))],
 [('paycheck', 'NOUN'), (('reasonably', 'X'), ('reasonably', 'ADV'))],
 [(',', '.'), (('though', 'ADP'), ('though', 'ADV'))],
 [('such', 'ADJ'), (('close', 'NOUN'), ('close', 'ADJ'))],
 [('acquiring', 'VERB'), (('more', 'ADV'), ('more', 'ADJ'))],
 [('*-1', 'X'), (('Funded', 'X'), ('Funded', 'VERB'))],
 [('from', 'ADP'), (('Tokio', 'X'), ('Tokio', 'NOUN'))],
 [('medical', 'ADJ'), (('protocols', 'X'), ('protocols', 'NOUN'))],
 [('on', 'ADP'), (('preventative', 'X'), ('preventative', 'ADJ'))],
 [('it', 'PRON'), (('existed', 'X'), ('exi

### Solve the problem of unknown words

### Part-1

In [35]:
# Based on the analysis of incorrectly tagged cases, there are quite a few unknown words, all of which are 
# getting a random tag assigned. And the random tag turns out to be the first tag in the set of tags obtained by 
# consolidating POS tags of all words in the training data set

### Why does the Viterbi algorithm choose a random tag on encountering an unknown word

In [36]:
# The random tag turns out to be the first tag in the set of tags obtained by 
# consolidating POS tags of all words in the training data set

In [37]:
# The analysis also presents the following key findings:
#     1. Emission probability for all such unknown words is 0, because of this when multiplied, 
#        transition probability is ignored
#     2. Hence, a random POS tag as explained above gets assigned
#     3. For this reason, we will ignore emission probability and base the assignment of POS tag on transition 
#        probability only

In [38]:
# The above implementation of Viterbi algorithm is modified slightly to consider the discussed changes into
# consideration

In [39]:
# Viterbi Heuristic
def Viterbi_transition(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        trans_prob = []
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities

            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p  
            p.append(state_probability)
            trans_prob.append(transition_p)
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 

        # Check if the emission probability for the word against each POS tag is zero, this means that word
        # is missing from the training set, in such a case, ignore the emission probability and consider
        # only the transition probability
         
        if pmax == 0:
            pmax = max(trans_prob)
            state_max = T[trans_prob.index(pmax)]

        state.append(state_max)
    return list(zip(words, state))    

In [40]:
# Execute the algorithm to find the new accuracy

In [41]:
start = time.time()
tagged_seq = Viterbi_transition(test_tagged_words[:])
end = time.time()
difference = end-start

In [42]:
print(tagged_seq)

[('For', 'ADP'), ('the', 'DET'), ('Agency', 'NOUN'), ('for', 'ADP'), ('International', 'NOUN'), ('Development', 'NOUN'), (',', '.'), ('appropriators', 'NOUN'), ('approved', 'VERB'), ('$', '.'), ('200', 'NUM'), ('million', 'NUM'), ('*U*', 'X'), ('in', 'ADP'), ('secondary', 'ADJ'), ('loan', 'NOUN'), ('guarantees', 'NOUN'), ('under', 'ADP'), ('an', 'DET'), ('expanded', 'VERB'), ('trade', 'VERB'), ('credit', 'NOUN'), ('insurance', 'NOUN'), ('program', 'NOUN'), (',', '.'), ('and', 'CONJ'), ('total', 'ADJ'), ('loan', 'NOUN'), ('guarantees', 'NOUN'), ('for', 'ADP'), ('the', 'DET'), ('Overseas', 'NOUN'), ('Private', 'ADJ'), ('Investment', 'NOUN'), ('Corp.', 'NOUN'), ('are', 'VERB'), ('increased', 'VERB'), ('*-3', 'X'), ('by', 'ADP'), ('$', '.'), ('40', 'NUM'), ('million', 'NUM'), ('*U*', 'X'), ('over', 'ADP'), ('fiscal', 'ADJ'), ('1989', 'NUM'), ('as', 'ADP'), ('part', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('same', 'ADJ'), ('Poland', 'NOUN'), ('package', 'NOUN'), ('.', '.'), ('The', 'DET'), 

### Evaluating tagging accuracy

In [43]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 

In [44]:
accuracy = len(check)/len(tagged_seq)
print(accuracy)

0.9388637703951248


### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

In [45]:
# As compared to 91.48% accuracy, we have now obtained 93.88% accuracy which is an improvement almost 2.5%

In [46]:
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1]]

In [47]:
incorrect_tagged_cases

[[('expanded', 'VERB'), (('trade', 'VERB'), ('trade', 'NOUN'))],
 [('Overseas', 'NOUN'), (('Private', 'ADJ'), ('Private', 'NOUN'))],
 [('settled', 'VERB'), (('pre-1917', 'X'), ('pre-1917', 'ADJ'))],
 [('the', 'DET'), (('purchasing', 'NOUN'), ('purchasing', 'VERB'))],
 [('paycheck', 'NOUN'), (('reasonably', 'NOUN'), ('reasonably', 'ADV'))],
 [(',', '.'), (('though', 'ADP'), ('though', 'ADV'))],
 [('such', 'ADJ'), (('close', 'NOUN'), ('close', 'ADJ'))],
 [('acquiring', 'VERB'), (('more', 'ADV'), ('more', 'ADJ'))],
 [('from', 'ADP'), (('Tokio', 'DET'), ('Tokio', 'NOUN'))],
 [('on', 'ADP'), (('preventative', 'DET'), ('preventative', 'ADJ'))],
 [('$', '.'), (('20.5', 'NOUN'), ('20.5', 'NUM'))],
 [('become', 'VERB'), (('polarized', 'X'), ('polarized', 'VERB'))],
 [(',', '.'), (('so', 'ADV'), ('so', 'ADP'))],
 [('a', 'DET'), (('middle', 'NOUN'), ('middle', 'ADJ'))],
 [('.', '.'), (('Though', 'NOUN'), ('Though', 'ADP'))],
 [('totaled', 'VERB'), (('154.2', 'X'), ('154.2', 'NUM'))],
 [('Since', 

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

In [48]:
# Following are some words that were incorrectly tagged (due to the word missing in training data) &
# due to which it got randomly assigned "X" tag:
    
# 1. Overseas - NOUN
# 2. Unemployment - NOUN
# 3. paycheck - NOUN
# 4. Funded - VERB
# 5. existed - VERB

# were all incorrectly tagged into X which is the first POS tag in the set defined on the POS tags of all words in the
# training data set

### Part 2:

In [49]:
# We notice that there is a good improvement in the accuracy. However, upon further analysis, there are still certain
# words getting an incorrect POS tag assigned due to a misleading transition probability

In [50]:
# To handle, these inaccuracies, we will define a few rules using the morphological structure of words which provide
# guidance on what the potential POS tag could be for the word

### Which tag class do you think most unknown words belong to?

In [51]:
# It can be observed that most unknown words are nouns that have a starting letter in upper case. According
# to this, observation of training data and some of the other rules, the following function has been defined

In [52]:
def apply_morphological_rules(text):
    # Initialize a list where the updated POS tags can be captured
    new_list = []
    
    for i, tup in enumerate(text):

        # If the word is a 0, 1, 2, 3 which was assigned with a POS tag X, do not change it
        if tup[0] == '0' or tup[0] == '1' or tup[0] == '2' or tup[0] == '3':
            new_list.append((tup[0], 'X'))
            continue
        
        # If the first character of the word is an *, assign it with a POS tag of X
        if tup[0][0] == '*':
            new_list.append((tup[0], 'X'))
            continue
        
        # For an imperative verb such as SHOW which the algorithm consistently assigns incorrectly, 
        # assign it as a very
        if tup[0] in ['Show', 'Use', 'Give', 'Eat', 'Sleep']:
            new_list.append((tup[0], 'VERB'))
            continue
            
        # The word THEY is a pronoun regardless of the context in which it is used    
        if tup[0].lower() == 'they':
            new_list.append((tup[0], 'PRON'))
            continue
            
        # The word INSTEAD is a pronoun regardless of the context in which it is used
        if tup[0].lower() == 'instead':
            new_list.append((tup[0], 'ADV'))
            continue            
        
        # The word p.m is a pronoun regardless of the context in which it is used
        if tup[0] == 'p.m':
            new_list.append((tup[0], 'ADV'))
            continue 
        
        # If there is a 2 digit number preceded by a single quote, assign the NUM tag
        # Examples are '86 and '88
        if re.search("^'\d{2}", tup[0]) != None:
            new_list.append((tup[0], 'NUM'))
            continue
        
        # For patterns such as whole numbers of ddd, ddd, ddd or d:dd, assign the NUM tag
        if re.search('^\d+(\.|:|,|-)?\d{0,3}(,)?\d{0,3}$', tup[0]) != None:
            new_list.append((tup[0], 'NUM'))
            continue
        
        # For examples such as 1st, 2nd, 3rd, 4th, assign ADJ as the POS tag    
        if re.search('^\d+(nd|rd|st|th)$', tup[0]) != None:
            new_list.append((tup[0], 'ADJ'))
            continue
        
        # According to the language rules, most adverbs end with "ly"
        if re.search('^\w*ly$', tup[0]):    
            new_list.append((tup[0], 'ADV'))
            continue                 
        
        # According to the language rules, most verbs if not all end with "ed", such as jumped, turned
        if (re.search('^[\w]*ed$', tup[0]) and len(tup[0]) > 4):
            new_list.append((tup[0], 'VERB'))
            continue

        # handles phrases such as heavy-duty, direct-investment most of which are adjectives with very few exceptions
        if re.search('\w+-[a-z]+', tup[0]) != None:
            new_list.append((tup[0], 'ADJ'))
            continue
            
        # Words that are alphabetic and start with a capital letter are typically nouns
        # We want to ignore words such as It, By which are not nouns      
        if re.search('^[A-Z]+[a-z]*', tup[0]) and len(tup[0]) > 3:
            new_list.append((tup[0], 'NOUN'))
            continue
            
        new_list.append(tup)        
                            
    return(new_list)

Please note that the new function is being invoked on the output of the modified Viterbi algorithm

In [53]:
new_tagged_seq = apply_morphological_rules(tagged_seq)

### Evaluating tagging accuracy

In [54]:
# accuracy
check = [i for i, j in zip(new_tagged_seq, test_run_base) if i == j] 

In [55]:
accuracy = len(check)/len(new_tagged_seq)
print(accuracy)

0.9496756437979162


### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

In [56]:
# As compared to 91.48% accuracy, we have now obtained ~95% accuracy which is an improvement of almost 3.5%

In [57]:
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(new_tagged_seq, test_run_base)) if j[0]!=j[1]]

In [58]:
incorrect_tagged_cases

[[('expanded', 'VERB'), (('trade', 'VERB'), ('trade', 'NOUN'))],
 [('settled', 'VERB'), (('pre-1917', 'X'), ('pre-1917', 'ADJ'))],
 [('the', 'DET'), (('purchasing', 'NOUN'), ('purchasing', 'VERB'))],
 [('the', 'DET'), (('weekly', 'ADV'), ('weekly', 'ADJ'))],
 [(',', '.'), (('though', 'ADP'), ('though', 'ADV'))],
 [('such', 'ADJ'), (('close', 'NOUN'), ('close', 'ADJ'))],
 [('to', 'PRT'), (('Japanese', 'NOUN'), ('Japanese', 'ADJ'))],
 [('acquiring', 'VERB'), (('more', 'ADV'), ('more', 'ADJ'))],
 [('more', 'ADJ'), (('American', 'NOUN'), ('American', 'ADJ'))],
 [('$', '.'), (('1', 'X'), ('1', 'NUM'))],
 [('follow', 'VERB'), (('Japanese', 'NOUN'), ('Japanese', 'ADJ'))],
 [('on', 'ADP'), (('preventative', 'DET'), ('preventative', 'ADJ'))],
 [(',', '.'), (('so', 'ADV'), ('so', 'ADP'))],
 [('a', 'DET'), (('middle', 'NOUN'), ('middle', 'ADJ'))],
 [('.', '.'), (('Though', 'NOUN'), ('Though', 'ADP'))],
 [('.', '.'), (('Since', 'NOUN'), ('Since', 'ADP'))],
 [('Since', 'ADP'), (('chalk', 'DET'), ('

### Following are the cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

In [59]:
# Following words were incorrectly tagged as below

# 1. Private - NOUN was tagged as ADJ
# 2. Tokio - NOUN was tagged as DET
# 3. 20.5 - NUM was tagged as NOUN
# 4. touched - VERB was tagged as NOUN
# 5. reasonably - ADV was tagged as NOUN

# all of which got rectified

In [60]:
# Following is a demonstration of a few test sentences which go through the modified Viterbi as well as application of
# morphological rules

In [61]:
Test_sentences = []
Test_sentences.append('Android is a mobile operating system developed by Google.')
Test_sentences.append('Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.')
Test_sentences.append("Google and Twitter made a deal in 2015 that gave Google access to Twitter's firehose.")
Test_sentences.append('Twitter is an online news and social networking service on which users post and interact with messages known as tweets.')
Test_sentences.append('Before entering politics, Donald Trump was a domineering businessman and a television personality.')
Test_sentences.append('The 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years.')
Test_sentences.append('This is the first World Cup to be held in Eastern Europe and the 11th time that it has been held in Europe.')
Test_sentences.append('Show me the cheapest round trips from Dallas to Atlanta')
Test_sentences.append('I would like to see flights from Denver to Philadelphia.')
Test_sentences.append('Show me the price of the flights leaving Atlanta at about 3 in the afternoon and arriving in San Francisco.')
Test_sentences.append('NASA invited social media users to experience the launch of ICESAT-2 Satellite.')

In [62]:
for sent in Test_sentences:
    print(apply_morphological_rules(Viterbi_transition(word_tokenize(sent))))

[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.')]
[('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'DET'), ('since', 'ADP'), ('2011', 'NUM'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NUM'), ('.', '.')]
[('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NUM'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'NOUN'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'NOUN'), ("'s", 'PRT'), ('firehose', 'VERB'), ('.', '.')]
[('Twitter', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('online', 'NOUN'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET'),

### Final Summary

In [63]:
# The default algorithm was unable to handle a number of Unknown words, i.e. words that were not in the training set
# This is likely the situation that we will encounter in most scenarios.

# By applying the following 2 techniques:
# 1. Use of transition probabilities where emission probability is missing
# 2. Applying common morphological rules,

# we were able to establish that the prediction accuracy went up